In [ ]:
import qiskit.quantum_info as qi
import math
import matplotlib.pyplot as plt
import numpy as np
from qiskit import QuantumCircuit, Aer, transpile, assemble, execute,QuantumRegister, ClassicalRegister
from numpy.random import randint
from numpy import pi
from time import time
import scipy as sc
import scipy.linalg as la
import json

import warnings
warnings.filterwarnings('ignore')

# define noise model for simulation
from qiskit.providers.aer.noise import NoiseModel, amplitude_damping_error, depolarizing_error, coherent_unitary_error,phase_damping_error
from scipy.optimize import curve_fit, minimize
from qiskit.quantum_info import process_fidelity, PTM, diamond_norm

def unitary_error_rx(theta):  #exp(-i theta/2 X)
    return np.array([[np.cos(theta/2), -1j*np.sin(theta/2)], [-1j*np.sin(theta/2), np.cos(theta/2)]])

def unitary_error_rxx(theta):          #exp(-i theta/2 X\otimes X)
    return np.cos(theta/2)*np.eye(4)-1j*np.sin(theta/2)*np.array([[0,0,0,1],[0,0,1,0],[0,1,0,0],[1,0,0,0]])

def unitary_error_rz(theta):  #exp(-i theta/2 Z)
    return np.array([[np.exp(-1j*theta/2), 0], [0, np.exp(1j*theta/2)]])

def unitary_error_rzz(theta):          #exp(-i theta/2 Z\otimes Z)
    return np.cos(theta/2)*np.eye(4)-1j*np.sin(theta/2)*np.array([[1,0,0,0],[0,-1,0,0],[0,0,-1,0],[0,0,0,1]])

def unitary_error_ry(theta):  #exp(-i theta/2 Y)
    return np.array([[np.cos(theta/2), -np.sin(theta/2)], [np.sin(theta/2), np.cos(theta/2)]])

def unitary_error_ryy(theta):          #exp(-i theta/2 Y\otimes Y)
    return np.cos(theta/2)*np.eye(4)-1j*np.sin(theta/2)*np.array([[0,0,0,-1],[0,0,1,0],[0,1,0,0],[-1,0,0,0]])

def fsim_error(theta,phi):
    mat= [[1, 0, 0, 0],[0, np.cos(theta), -1j*np.sin(theta),0], [0,-1j*np.sin(theta),np.cos(theta),0],[0,0,0,np.exp(1j*phi)]]
    return mat

# generate unitary noise
def get_noise_model(ps=0.001,angle=[0,0.01]):
    noise_model=NoiseModel()
    
    error_ad= amplitude_damping_error(ps)
    error_pd= phase_damping_error(ps)
    error_st= error_pd.compose(error_ad)
    
    error_st_2q= error_st.tensor(error_st)
    
    error_u_mat= fsim_error(*angle)
    error_u = coherent_unitary_error(error_u_mat)
    
    error_1q= error_st
    error_2q= error_u.compose(error_st_2q)
    noise_model.add_all_qubit_quantum_error(error_1q, ['t','s','sdg','h','u1','u2','u3','I','X','Y','Z','ry','rz','rx'])
    noise_model.add_all_qubit_quantum_error(error_2q, ['cz','cx','fsim'])
    
    return noise_model,error_2q

# simulate circuits with given noise model
def noise_sim(circ_list,noise_model,shots=10**4):
    result = execute(circ_list, backend=Aer.get_backend('qasm_simulator'),noise_model=noise_model, shots=shots,optimization_level=0).result()
    count_list=[result.get_counts(circ) for circ in circ_list]
    exp_list=[count.get('00',0)/shots for count in count_list]
    
    return exp_list

# actual process infidelity and stochastic infidelity of noise 
def act_error(error_1q,target_mat):
    pf= process_fidelity(error_1q)
    noisy_gate=error_1q.compose(coherent_unitary_error(target_mat))
    ptm=PTM(noisy_gate)
    evals, evecs=np.linalg.eig(ptm)
    stof= np.sqrt(np.mean(np.square(np.abs(evals))))
    
    return 1-pf, 1-stof


#define matrix pencil data processing technique; this code is from the paper [spectral quantum tomography] with some modifications.
def matrix_pencil(time_series_data,L ,N_poles,cutoff=10**(-2)):
        """Computes a decomposition into exponentially decaying oscillations of a given time series.
        Input: time_series_data: a list of floats corresponding to the state of the system at fixed time intervals.
                NOTE: It is important this timeseries starts at t=0 (k=0), the method as implemented can't deal with timeshifts and may fail quietly
               L: A matrix pencil parameter. Set between 1/2 and 2/3 of len(time_series_data) 
               N_poles: The number of poles the data can be decomposed into. Choosing this number too small will lead to bad fits so act with care.
               cutoff: a cut-off for the smallest possible relative amplitude a poles can contribute with, standard is 10^(-2).
        Ouput: poles: a scipy array of the poles (the oscillating bits).
                amplitudes: a scipy array of amplitudes corresponding to the poles.
                
                """   
        #Compute the length of the data series and store it in N
        N = len(time_series_data)

        #Compute the Hankel matrix of the data
        Y = sc.matrix([time_series_data[i:L+i+1] for i in range(0,N-L)]) 
        
        #Take the singular value decomposition of the data Hankel matrix
        U,S,Vh = sc.linalg.svd(Y) 
        Vh =sc.matrix(Vh)
        U = sc.matrix(U)

        #The ESPRIT method includes a filter step that gets rid of small singular values.
        # Since for us the number of poles is known I just retain the N_poles largest singular values and corresponding right eigenspace.
        #If N_poles is so large that it starts to include nonsense values we let the cutoff parameter set the number of relevant poles instead.
        #We choose to retain only the singular values s such that s>s_max * cutoff where s_max is the largest singualr value
        Scutoff = S[S>cutoff*S[0]]
        if len(Scutoff)<N_poles:

            Sprime = sc.matrix([[S[i] if i==j else 0 for i in range(len(Scutoff))] for j in range(N-L)])
            Vhprime = Vh[0:len(Scutoff),:]
        else:
            Sprime = sc.matrix([[S[i] if i==j else 0 for i in range(N_poles)] for j in range(N-L)])
            Vhprime = Vh[0:N_poles,:]
        
        #Compute the shifted matrices for the matrix pencil.
        Vhprime1 = Vhprime[:,0:-1]
        Vhprime2 = Vhprime[:,1:]
        
        #Compute the solution of the matrix pencil (via SVD)
        Y = la.pinv(Vhprime1.H)*Vhprime2.H
        poles, vecs = sc.linalg.eig(Y)
        for i in range(len(poles)):
            amp= np.abs(poles[i])
            if amp>1:
                poles[i]= poles[i]/amp
        
        #Compute the amplitudes by least squares optimization
        Z = sc.matrix([poles**k for k in range(N)])
        amplitudes = la.lstsq(Z,sc.matrix(time_series_data).transpose())
        ampls = sc.array([a[0] for a in amplitudes[0]])
        amp_max= np.max(np.abs(ampls))
        
        polesp=[]
        amplsp=[]
        for i in range(len(poles)):
            if np.abs(ampls[i])/amp_max>0.1:
                polesp.append(poles[i])
                amplsp.append(ampls[i])
                
        #return the poles and amplitudes as scipy arrays
        return polesp, amplsp, amplitudes, S

# processing the estimates from matrix pencil method, return the phase error, diagonal entries of pure noise
def mp_est(poles,target_phase=1/3*pi,nrep=1):
    amps= np.abs(poles)
    phases= np.angle(poles)
    res=[]
    target_phase_p= (target_phase*nrep)%(2*pi)
    if target_phase_p-pi> 10**-4:
        target_phase_p= target_phase_p-2*pi
     
    
    if np.abs(target_phase_p) > 10**-6:
        phase_difs= np.abs(phases-target_phase_p)
        angle_index= np.argmin(phase_difs)
        
        phase_difs2= np.abs(np.abs(phases)-np.abs(target_phase_p))
        angle_error2= np.min(phase_difs2)
        index_others= np.argwhere(phase_difs2>angle_error2)[:,0]

        res.append((phases-target_phase_p)[angle_index]/nrep)
        res.append((amps[angle_index])**(1/nrep))
        for index in index_others:
            res.append((poles[index])**(1/nrep))
        res.append(target_phase)  
    else:
        phase_difs= np.abs(phases)
        angle_error= np.max(phase_difs)
        res.append(angle_error/nrep)
        for pole in poles:
            res.append(pole**(1/nrep))
        res.append(0)    
    return res

# input: output of function 'mp_est'
#output: process infidelity, stochastic infidelity, theta angle error, phi angle error
def result_mp(mp_est_list,theta,phi,ndeg=0):
    dim= 4
    dim_ts= dim + ndeg**2-ndeg        #the dimension of trivial eigen-subspace with eigenvalue 1
    dim_ns= dim**2-dim_ts         # the dimension of non-trivial subspace
    ns_list=[]              
    ts_list= []
    angle_list=[]
    
    for mp_est in mp_est_list:
        if np.abs(mp_est[-1])>10**-8:
            ns_list.append(mp_est[1]*np.exp(1j*mp_est[0]))
            ns_list.append(mp_est[1]*np.exp(-1j*mp_est[0]))
            for x in mp_est[2:-1]:
                ts_list.append(x)
        else:
            for x in mp_est[1:-1]:
                ts_list.append(x)
        angle_list.append(mp_est[0])
    
    f_ns_mean= np.abs(np.mean(ns_list))
    f_ts_mean= np.abs(np.mean(ts_list))
    f_mean= (dim_ns*f_ns_mean+ dim_ts*f_ts_mean)/dim**2
    
    u_ns_mean= np.mean(np.square(np.abs(ns_list)))
    u_ts_mean= np.mean(np.square(np.abs(ts_list)))
    u_mean= (dim_ns*u_ns_mean+ dim_ts*u_ts_mean)/dim**2
    
    if np.abs(np.abs(theta)-pi)< 10**-6:
        theta_error =(angle_list[3]-angle_list[4])/2
        phi_error= -1*(angle_list[3]+angle_list[4])/2
    else:
        theta_error= np.mean(np.abs([angle_list[1],angle_list[2]]))* np.sign(angle_list[1])
        if np.abs(np.abs(phi)-pi)< 10**-6:
            phi_error= -1*(angle_list[3]+angle_list[4])/2
        else:
            phi_error= angle_list[0]
    
    return [1-f_mean, 1-np.sqrt(u_mean),theta_error, phi_error]


# for fsim gates with non-degenerate spectrum, i.e., both theta and phi are not equal to zero.
class Csb_fsim:
    def __init__(self, theta, phi):
        self.theta = theta
        self.phi = phi
        self.mat= self.fsim_mat()
        self.eigval_list= [1,np.exp(1j*phi),np.exp(1j*theta),np.exp(-1j*theta)]
        self.eigvec_list= [np.array([1,0,0,0]), np.array([0,0,0,1]),1/np.sqrt(2)*np.array([0,1,-1,0]),1/np.sqrt(2)*np.array([0,1,1,0])]

    def fsim_mat(self):
        mat= [[1, 0, 0, 0],[0, np.cos(self.theta), -1j*np.sin(self.theta),0], 
              [0,-1j*np.sin(self.theta),np.cos(self.theta),0],[0,0,0,np.exp(1j*self.phi)]]
        return mat

    def benchmark_circ(self,len_list,mode='01',nrep=1):
        fsim_op= qi.Operator(self.mat)
        circ_list=[]
        qc_ini= QuantumCircuit(2,2)  #initial state preparation for qiskit
        qc_ini.initialize(1/np.sqrt(2)*(self.eigvec_list[int(mode[0])]+self.eigvec_list[int(mode[1])]))
        qc_ini = transpile(qc_ini,basis_gates=['h','t','s','sdg','x','y','z', 'cx','u1', 'u2', 'u3'],optimization_level=3)
        qc_ini.barrier()
        qc_meas= qc_ini.inverse()

        for lc in len_list:
            qc_rep= QuantumCircuit(2,2)
            for k in range(lc*nrep):
                qc_rep.unitary(fsim_op,[0,1], label='fsim')
                
            qc=qc_ini.combine(qc_rep)
            qc= qc.combine(qc_meas)
            qc.measure([0,1],[0,1])
            circ_list.append(qc)
        return circ_list
     
    # the full list of circuits for csb
    def full_circs(self,len_max=50,nrep_list=[1 for i in range(6)]):
        self.nrep_list =nrep_list
        mode_list= ['01','02','03','12','13','23']
        len_list=[i for i in range(0,len_max+1)]
        circs_list=[]
        for k,mode in enumerate(mode_list):
            circs_list.append(self.benchmark_circ(len_list,mode=mode,nrep=nrep_list[k]))
        return circs_list
    
    # input: the list of lists of measured probabilities of circuits;
    # output: [process infidelity, stochastic infidelity,theta angle error, phi angle error]; diagonal entries of the pure noise channel
    def result(self,data):
        mode_list= ['01','02','03','12','13','23']
        target_phases=[]
        num_modes= len(mode_list)
        index_theta=[]
        index_phi=[]
        for k,mode in enumerate(mode_list):
            phase=np.angle(self.eigval_list[int(mode[1])])-np.angle(self.eigval_list[int(mode[0])])
            phase = phase %(2*pi)
            if phase-pi>10**-4:
                phase= phase-2*pi
            target_phases.append(phase)
            
        ndeg=0
        if self.theta==self.phi:
            ndeg=2
            
        mp_est_list=[]
        for i in range(num_modes):
            max_len= len(data[i])
            poles, amps, amplitudes,S = matrix_pencil(data[i],L= math.ceil(max_len*3/5),N_poles=4,cutoff=10**-2)
            mp_est_list.append(mp_est(poles,target_phase=target_phases[i],nrep= self.nrep_list[i]))
        results = result_mp(mp_est_list,theta=self.theta,phi=self.phi,ndeg=ndeg)
            
        return results,mp_est_list
    
    def simulation(self,noise_model,len_max=50,shots=10**4,nrep_list=[1 for i in range(6)]):
        circs=self.full_circs(len_max,nrep_list=nrep_list)
        data=[]
        for circ_list in circs:
            plist=noise_sim(circ_list,noise_model=noise_model,shots=shots)
            data.append(plist)
        res, spectrum=self.result(data)
        return res,spectrum

In [ ]:
ts= time()
ps_list=[0.001,0.002,0.005,0.007,0.01,0.02,0.05,0.07,0.1]
L_max=50
shots=10**4
act_error_list=[]
est_sto=[]
target_mat= fsim_error(pi/4,pi/2)
for ps in ps_list:
    noise_model,error_2q= get_noise_model(ps=ps,angle=[-0.01,-0.02])
    act_error_list.append(act_error(error_2q,target_mat=target_mat))
    for k in range(10):
        fsim_csb= Csb_fsim(pi/4,pi/2)
        res, sp=fsim_csb.simulation(noise_model=noise_model,len_max=L_max,shots=shots)
        est_sto.append(res)
te= time()
print(te-ts)

In [ ]:
ts= time()
angle_list=[0.001,0.002,0.005,0.007,0.01,0.02,0.05,0.07,0.1]
L_max=100
shots=10**4
act_uni_error_list=[]
est_uni=[]
target_mat= fsim_error(pi/4,pi/2)
for angle in angle_list:
    noise_model,error_2q= get_noise_model(ps=0.001,angle=[angle,2*angle])
    act_uni_error_list.append(act_error(error_2q,target_mat=target_mat))
    for k in range(10):
        fsim_csb= Csb_fsim(pi/4,pi/2)
        res, sp=fsim_csb.simulation(noise_model=noise_model,len_max=L_max,shots=shots)
        est_uni.append(res)
te= time()
print(te-ts)